In [1]:
import torch
import cvxpy as cvx
import numpy as np
from scipy.stats import chi2
import matplotlib.pyplot as pl

/home/mkallel/miniconda3/envs/bopt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [267]:
def get_bound(n_dim,p):

    dx = 0.001
    x = np.arange(0, n_dim,dx)

    cdf = np.cumsum(chi2.pdf(x, df=n_dim)*dx)
    bound = max(x[cdf<=p])

    print("bound",bound)
    return bound

get_bound(100,0.05/100)

bound 59.895


59.895

In [259]:
n_dim = 100
mu = np.random.rand(n_dim)
A = 0.25 * np.random.rand(n_dim,n_dim) +  0.25 *mu
A.dtype = np.float64
A = 1e-3* A
Corr = np.random.rand(n_dim) 
Corr = np.diag(Corr)

Sigma = A.T @ A 
Sigma_inv = np.linalg.inv(Sigma)


In [260]:

m = np.mean(np.abs(Sigma))
Sigma_b = Sigma / m
Sigma_inv = np.linalg.inv(Sigma_b)/ m
Sigma_inv2 = np.linalg.inv(Sigma)

P_b = np.linalg.cholesky(Sigma_b)
P_b_inv = np.linalg.inv(P_b)
P_p = P_b_inv / np.sqrt(m)
print("INVERSE ERR",np.mean(np.abs(P_p.T@P_p-Sigma_inv)))

INVERSE ERR 0.01207757922660294


In [261]:
eigvals,P = np.linalg.eigh(Sigma)
D = np.diag(eigvals)
D_sqrt = np.diag(np.sqrt(eigvals))

D_inv = np.diag(1/eigvals)
D_inv_sqrt = np.diag(np.sqrt(1/eigvals))
#P_p = D_sqrt_inv@(P.T)
P_p = (P @ D_inv_sqrt).T
#print("Error",np.max(P@D_sqrt@D_sqrt@P.T-Sigma))
print("Error",np.mean(np.abs(P_p.T@P_p-Sigma_inv)))

Error 0.011926387469739529


In [243]:
p = 0.05
d = 10
get_bound(d,p/d)

bound 2.1550000000000002


2.1550000000000002

In [189]:
Sigma = torch.tensor(Sigma)
L_upper = torch.linalg.cholesky(Sigma,upper=True)
Sigma_inv = torch.cholesky_inverse(L_upper,upper=True)
Sigma_inv

/tmp/ipykernel_3720604/1656108993.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Sigma = torch.tensor(Sigma)


tensor([[ 2.0703e+10, -4.9161e+10,  8.3894e+10,  ...,  9.2684e+09,
         -4.6628e+09, -2.8921e+10],
        [-4.9161e+10,  2.4058e+11, -4.1555e+11,  ..., -6.0521e+10,
          1.1987e+10,  1.7468e+11],
        [ 8.3894e+10, -4.1555e+11,  7.4478e+11,  ...,  1.0359e+11,
         -1.5381e+10, -3.2328e+11],
        ...,
        [ 9.2684e+09, -6.0521e+10,  1.0359e+11,  ...,  1.9235e+10,
         -2.0322e+09, -4.5788e+10],
        [-4.6628e+09,  1.1987e+10, -1.5381e+10,  ..., -2.0322e+09,
          5.6224e+09,  3.2697e+09],
        [-2.8921e+10,  1.7468e+11, -3.2328e+11,  ..., -4.5788e+10,
          3.2697e+09,  1.5616e+11]], dtype=torch.float64)

In [107]:
import torch
Sigma = torch.tensor(Sigma)
Sigma_b = Sigma / torch.min(torch.abs(Sigma))
Sigma_inv = torch.linalg.inv(Sigma)
P = torch.linalg.cholesky(Sigma)
P_inv = torch.linalg.inv(P)
print("Err1",torch.max(torch.abs(P@P.T-Sigma)))
print("Err2",torch.max(torch.abs(P_inv.T@P_inv-Sigma_inv)))

eigvals,P = torch.linalg.eigh(Sigma)
D = torch.diag(eigvals)
D_inv = torch.diag(1/eigvals)
print("Err3",torch.max(torch.abs(P@D@P.T-Sigma)))
print("Err4",torch.max(torch.abs(P@D_inv@P.T-Sigma_inv)))






Err1 tensor(1.3235e-22, dtype=torch.float64)
Err2 tensor(0.5590, dtype=torch.float64)
Err3 tensor(8.4703e-22, dtype=torch.float64)
Err4 tensor(1.7846, dtype=torch.float64)


In [30]:
def sparsify_grad(mu,Sigma):

    Sigma_inv = np.linalg.inv(Sigma)
    n_dim = mu.shape[0]
    
    eigvals,P = np.linalg.eigh(Sigma)
    assert (eigvals >= 0).all()

    
    ### Square root of Sigma_inv

    D = np.diag(1/eigvals)
    D_sqrt = np.diag(np.sqrt(1/eigvals))
    P_p = P.T@D_sqrt 

    print("One",np.max(P@eigvals@(P.T)-Sigma))
    print("Distance",np.max(P_p@(P_p.T)-Sigma_inv))
    print("Distance",np.max((P_p.T)@(P_p)-Sigma_inv))

    
    ###################

    C = get_bound(n_dim,p=0.05)

    x = cvx.Variable((n_dim))
    objective = cvx.Minimize(cvx.norm(x,1))
    pb = cvx.Problem(objective,
                    [cvx.sum_squares(P_p@x -P_p@mu) <= C])

    pb.solve(verbose=False)

    rslt = x.value

    assert all(rslt<=mu)

    relative_diff = abs(rslt-mu)/mu

    plt.hist(relative_diff)

    return np.sum(relative_diff>0.9), np.sum(relative_diff<0.1),np.sum(relative_diff<0.5)

sparsify_grad(mu,Sigma)

In [17]:
%cd /home/mkallel/explo/
from src.gp.kernels import MyRBFKernel

n_dim = 6
kernel = MyRBFKernel(6,True)
theta = torch.rand(1,6,requires_grad=True)
kernel(theta,theta).evaluate()

theta_t2 = theta.clone().detach() ## hotfix otherwise 0 hessian
## this might be a cause of error, try to find method to compute using k(theta,theta)
hessian1 = -torch.autograd.functional.hessian(func=lambda theta : kernel(theta,theta_t2).evaluate(),
                                            inputs=(theta))


        

print(hessian1.squeeze())
        

/home/mkallel/explo
tensor([[1.4427, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0000, 1.4427, -0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, 1.4427, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, 1.4427, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, 1.4427, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, 1.4427]])


In [23]:
lengthscale = kernel.base_kernel.lengthscale.detach()
sigma_f = kernel.outputscale.detach()
hessian2 = (torch.eye(n_dim, device=lengthscale.device) / lengthscale ** 2) * sigma_f
print(hessian2.squeeze())

tensor([[1.4427, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.4427, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.4427, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.4427, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.4427, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.4427]])


In [1]:
%cd /home/mkallel/explo/

import logging
import logging.config
import os
from multiprocessing import Pool
from warnings import simplefilter

import gpytorch
import numpy as np
import torch
import threading
import wandb
from src.config import get_configs
from src.helpers import setup_experiment
from src.trainer import Trainer
import itertools


logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()

simplefilter(action='ignore', category=DeprecationWarning)
os.environ["WANDB_API_KEY"]="28996bd59f1ba2c5a8c3f2cc23d8673c327ae230"

def run(seed,
        env_name,
        kernel_name,
        manipulate_state,
        norm_grad,
        conf_grad,
        advantage_mean,
        adaptive_lr,
        lr ):

        #env_name = "CartPole-v1" ## Action kernel + State_norm looks very well for cartpole
        #env_name = "Swimmer-v4" ##  State_norm stabilizes training 
        #env_name = "Hopper-v2"
        #env_name = "HalfCheetah-v2"        
        #env_name = "Walker2d-v3"

        #kernel_name = "rbfstate" ## "rbf"
        #kernel_name = "rbf" ## "rbf"

        project_name = env_name+("RBF + Confidence Grad 2")
        run_name =  kernel_name +"_lr="+str(lr) +"_"+str(1 *manipulate_state)+ str(1 *norm_grad) + str(1 *conf_grad) + str(1 *advantage_mean)+str(1 *adaptive_lr) +"_"+ str(seed)
        env_config,policy_config,likelihood_config,kernel_config,mean_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name,
        use_ard=True,manipulate_state=manipulate_state,
        conf_grad=conf_grad,norm_grad=norm_grad,advantage_mean=advantage_mean,adaptive_lr=adaptive_lr,lr=lr,
        wandb_logger=True,project_name=project_name,run_name=run_name)

        model,objective_env,optimizer = setup_experiment(env_config,mean_config,kernel_config,likelihood_config,policy_config,optimizer_config,
                                        seed=seed)

        trainer = Trainer(model,objective_env,optimizer,**trainer_config)
        trainer.run()



if __name__ == '__main__':

        
        wandb.require("service")
        wandb.setup()  

        env_name = ["Swimmer-v4"]
        #env_name = ["CartPole-v1"]
        kernel_name = ["rbf"]
        manipulate_state = [False]
        conf_grad = [True] ##run this for rbf
        norm_grad = [True]
        advantage_mean = [True]
        adaptive_lr = [False]
        lr = [0.5]
        
        n= 10
        np.random.seed(42)
        seeds = np.random.randint(low=0,high=2**30,size=(n,))

        for config in itertools.product(*[env_name,kernel_name,manipulate_state,norm_grad,conf_grad,advantage_mean,adaptive_lr,lr]):

            
                seeds = [ int(i) for i in seeds]

                with Pool(processes=n) as p:
                        args = [(seed,*config) for seed in seeds]
                        p.starmap(run, args)

                #run(*(0,*config))

        

/home/mkallel/explo


/home/mkallel/miniconda3/envs/bopt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
MLPPPPPPPPP SETUPMLPPPPPPPPP SETUPMathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
MLPPPPPPPPP SETUPMLPPPPPPPPP SETUP  MLPPPPPPPPP SETUPMLPPPPPPPPP SETUP  1616 16 
16
MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]

1616MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
MLP LEEEEENMLP LEEEEEN
MLP LEEEEEN

MLPPPPPPPPP SETUPMLPPPPPPPPP SETUP MLP LEEEEENMathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
  MLP LEEEEEN MLP LEEEEENMathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
 1616MLPPPPPPPPP SETUP 16  1616

 MLPPPPPPPPP SETUP
1616
16
16
MLP LEEEEEN 
MLP LEEEEEN

  16MLP LEEEEEN16 16


16MLP LEEEEEN 
16
Using ard_num_dims = 16Using ard_num_dims = 16

Using ard_num_dims = 16
 Gibo wi

wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin


/home/mkallel/miniconda3/envs/bopt/lib/python3.9/site-packages/torch/autograd/__init__.py:275: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/mkallel/miniconda3/envs/bopt/lib/python3.9/site-packages/gpytorch/lazy/lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATe

...

0.0


0.0
rbf_lr=0.5_01110_423734972 : n_samples : 594


Mean MAE,▃▁▃▃▃▃▇█▆▇▅▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁
R1,▁▇▆▇▇▇▅▄▆▆▇██▇▇▇▇▇▇▇▇▇▇▇▆▇▇▆█▆▆▇▇▇
R2,▁█▇███▅▄▇▆▇██▇██████▇▇█▇▇█▇▆█▇▇▇▇▇
acq_value (after finish),▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
action_distance_grad,▁▃▃▂▃▆▅▅▇▅▆█▇▄▅▆▆▇▆▃▆▇▆▇▅▇▄▄█▅▃▄▆▄
action_distance_to_local,▁▁▂▃▅▄▃▃▆▄▃▄▂▇▄▅▄▅▅▇▇▅▃▆▅▇▆▄▅▅█▅▆▆▇▃▆▇▇▇
covar_lengthscale mean,▁▁████████████████████████████████
covar_lengthscale std,▁▁█▇█▃█▅▄▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
covar_output_scale,██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episode_length,▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁
fraction_sparse,▁▁▂▂▂▅█▅▇▅▇▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


0.0


Mean MAE,▁▁▂▄▄▅█▆▆▄▂▂▂▃▂▁▁▂▁▁▁▁▁▁▁▂▂▂▁▂▂▂▁▁
R1,▆▇▂▁▆▆▂▄▄▅▄▅▅▄▆▆▇▄▇▅▅▆▆▅▆█▆▇█▇▆▇▆▇
R2,██▃▄█▇▁▅▄▄▄▅▆▅▆▆▇▅▇▆▅▇▇▆▇▇▆▇██▆▇▇▇
acq_value (after finish),██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
action_distance_grad,▁▂▅▄▅▇▆▄▅▄▇▆▆▇▅▅▅▄▇▅▅█▅▆▆█▆▆▆█▆▇▇▇
action_distance_to_local,▁▁▁▅▄▃▅▆▄▄▅▇█▇▅▅▇▇▅▄▅█▃▃▅▅▄▄█▆▅▅█▄▅▄▂▄▆▅
covar_lengthscale mean,▁▁▇█▇█████████████████████████████
covar_lengthscale std,▁▁▆▄█▆▅▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
covar_output_scale,██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episode_length,▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁
fraction_sparse,▁▁▃▄▄▇█▆▄▂▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▂▁▃▁▂▁▁▁▁
